# Yankees - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Jan 12, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '52.44.171.130' 
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = 'ThisIsQA123' 
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 53 # Yankees

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrYankees].[ds].[getPropensityEventScoring] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,dimEventId,inMarket,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,credits_after_refund,NumberofGamesPerSeason,isNextGameBuyer
0,53,8,296,False,14244240,2017,Online Individual Game,30.0,2017-04-30,1.000000,6,TRUE,4380,68,33.27,3,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,79,0
1,53,13,278,False,23070972,2017,Online Individual Game,0.0,2017-04-13,0.714286,13,FALSE,3285,13,30.03,2,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,79,0
2,53,13,308,False,23070972,2017,Online Individual Game,0.0,1970-01-01,0.000000,27,FALSE,3285,118,30.03,2,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,79,0
3,53,120,332,False,2624031,2017,Online Individual Game,88.0,2017-07-04,1.000000,1,TRUE,2190,1,34.17,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,79,0
4,53,130,308,False,2492972,2017,Online Individual Game,55.0,2017-07-27,1.000000,46,TRUE,2190,46,53.42,1,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,79,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1075402 entries, 0 to 1075401
Data columns (total 44 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   lkupClientId            1075402 non-null  int64  
 1   dimCustomerMasterId     1075402 non-null  int64  
 2   dimEventId              1075402 non-null  int64  
 3   inMarket                1075402 non-null  bool   
 4   customerNumber          1075402 non-null  object 
 5   year                    1075402 non-null  int64  
 6   productGrouping         1075402 non-null  object 
 7   totalSpent              1075402 non-null  float64
 8   recentDate              1075402 non-null  object 
 9   attendancePercent       1075402 non-null  float64
 10  renewedBeforeDays       1075402 non-null  int64  
 11  isBuyer                 1075402 non-null  object 
 12  source_tenure           1075402 non-null  int64  
 13  tenure                  1075402 non-null  int64  
 14  di

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(1075402, 27)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextGameBuyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1075402 entries, 0 to 1075401
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   inMarket           1075402 non-null  bool   
 1   attendancePercent  1075402 non-null  float64
 2   renewedBeforeDays  1075402 non-null  int64  
 3   source_tenure      1075402 non-null  int64  
 4   tenure             1075402 non-null  int64  
 5   distToVenue        1075402 non-null  float64
 6   totalGames         1075402 non-null  int64  
 7   recency            1075402 non-null  int64  
 8   click_link         1075402 non-null  int64  
 9   open_email         1075402 non-null  int64  
 10  send_email         1075402 non-null  int64  
 11  isNextGameBuyer    1075402 non-null  int64  
dtypes: bool(1), float64(2), int64(9)
memory usage: 91.3 MB


### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,inMarket,attendancePercent,renewedBeforeDays,source_tenure,tenure,distToVenue,totalGames,recency,click_link,open_email,send_email,isNextGameBuyer
inMarket,1.00,-0.06,-0.10,-0.05,0.09,-0.21,0.08,0.04,0.02,0.00,-0.03,0.06
attendancePercent,-0.06,1.00,0.02,0.08,-0.18,0.07,0.18,-0.10,-0.17,-0.14,-0.28,-0.11
renewedBeforeDays,-0.10,0.02,1.00,0.14,0.07,0.07,0.02,-0.04,0.02,0.02,0.04,-0.05
source_tenure,-0.05,0.08,0.14,1.00,0.22,-0.17,0.11,0.04,0.14,0.15,0.25,0.08
tenure,0.09,-0.18,0.07,0.22,1.00,-0.18,0.10,0.07,0.22,0.25,0.33,0.09
distToVenue,-0.21,0.07,0.07,-0.17,-0.18,1.00,-0.04,-0.05,-0.09,-0.09,-0.14,-0.06
totalGames,0.08,0.18,0.02,0.11,0.10,-0.04,1.00,0.06,0.01,0.00,-0.04,0.24
recency,0.04,-0.10,-0.04,0.04,0.07,-0.05,0.06,1.00,0.10,0.11,0.10,0.26
click_link,0.02,-0.17,0.02,0.14,0.22,-0.09,0.01,0.10,1.00,0.59,0.47,0.14
open_email,0.00,-0.14,0.02,0.15,0.25,-0.09,0.00,0.11,0.59,1.00,0.54,0.13


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (860322, 12)
Unseen Data For Predictions: (215080, 12)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [10]:
setup(
    data= df_train, 
    target="isNextGameBuyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "inMarket",
        "attendancePercent",
        "attendancePercent",
        "renewedBeforeDays",
        "source_tenure",
        "tenure",
        "distToVenue",
        "totalGames",
        "recency",
        "click_link",
        "open_email" ,
        "send_email"
    ]
)

,Description,Value
0,session_id,1164
1,Target,isNextGameBuyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(860322, 12)"
5,Missing Values,False
6,Numeric Features,11
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


(        inMarket  attendancePercent  renewedBeforeDays  source_tenure  tenure  \
 662528       0.0                0.0                0.0         1825.0    12.0   
 29702        1.0                0.0                1.0          365.0     1.0   
 295357       1.0                1.0                1.0         3650.0   443.0   
 101634       1.0                0.0              105.0         1095.0   105.0   
 591873       1.0                0.0               14.0         1825.0   411.0   
 ...          ...                ...                ...            ...     ...   
 116813       0.0                0.0              177.0         1460.0   177.0   
 134658       0.0                0.0               41.0         2920.0   655.0   
 711652       0.0                1.0              142.0         1460.0   142.0   
 269642       1.0                1.0               34.0         1825.0   449.0   
 300994       1.0                1.0               10.0         1460.0    10.0   
 
         distT

In [11]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9737,0.9468,0.3591,0.6997,0.4746,0.4625,0.4898,15.6500
lr,Logistic Regression,0.9675,0.8573,0.1106,0.5467,0.1838,0.1746,0.2355,18.8590


In [12]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9736,0.9483,0.3613,0.6945,0.4753,0.4631,0.4894
1,0.9739,0.9450,0.3591,0.7070,0.4763,0.4643,0.4926
2,0.9731,0.9448,0.3323,0.6971,0.4501,0.4381,0.4699
3,0.9739,0.9462,0.3652,0.7033,0.4808,0.4688,0.4954
4,0.9739,0.9488,0.3654,0.7015,0.4805,0.4685,0.4949
5,0.9734,0.9477,0.3597,0.6877,0.4723,0.4600,0.4857
6,0.9742,0.9469,0.3667,0.7143,0.4846,0.4728,0.5006
7,0.9743,0.9472,0.3742,0.7136,0.4909,0.4791,0.5055
8,0.9732,0.9488,0.3562,0.6827,0.4681,0.4558,0.4813
9,0.9734,0.9445,0.3513,0.6950,0.4667,0.4546,0.4827


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?